In [1]:
import os
import numpy as np
import cv2
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.losses import mean_squared_error

def build_dehaze_net(input_shape, output_channels):
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Decoder
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    up1 = UpSampling2D(size=(2, 2))(conv4)

    concat1 = Concatenate()([conv3, up1])
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(concat1)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)
    up2 = UpSampling2D(size=(2, 2))(conv5)

    concat2 = Concatenate()([conv2, up2])
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(concat2)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)
    up3 = UpSampling2D(size=(2, 2))(conv6)

    concat3 = Concatenate()([conv1, up3])
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(concat3)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

    # Output
    output = Conv2D(output_channels, 1, activation='sigmoid', padding='same')(conv7)

    model = Model(inputs=inputs, outputs=output)
    return model

# Load images from multiple folders
def load_images(image_folders, desired_shape):
    images = []
    for folder in image_folders:
        image_names = os.listdir(folder)
        for image_name in image_names:
            image_path = os.path.join(folder, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, desired_shape[::-1])  # Reverse the order for (width, height)
                images.append(image)
    return images

# Paths for foggy images and corresponding clear images in multiple folders
foggy_images_folders = ['C:/Users/rahei/OneDrive/Desktop/archive/Foggy', 'C:/Users/rahei/OneDrive/Desktop/archive/Foggy 2-20230724T025947Z-001','C:/Users/rahei/OneDrive/Desktop/archive/Foggy-20230724T025946Z-001']
clear_images_folders = ['C:/Users/rahei/OneDrive/Desktop/archive/Clear', 'C:/Users/rahei/OneDrive/Desktop/archive/Clear 2-20230724T025949Z-001','C:/Users/rahei/OneDrive/Desktop/archive/Clear-20230724T025952Z-001']

# Load foggy and clear images from multiple folders
desired_shape = (480, 640)  # Updated desired shape
foggy_images = load_images(foggy_images_folders, desired_shape)
clear_images = load_images(clear_images_folders, desired_shape)

# Convert lists to NumPy arrays
foggy_images = np.array(foggy_images)
clear_images = np.array(clear_images)

# Normalize images to [0, 1] range
foggy_images = foggy_images.astype('float32') / 255.0
clear_images = clear_images.astype('float32') / 255.0

# Build the dehazing model
input_shape = foggy_images.shape[1:]  # Shape of one input sample (excluding batch size)
output_channels = foggy_images.shape[-1]  # Number of output channels (RGB: 3)
model = build_dehaze_net(input_shape, output_channels)

# Perceptual loss using VGG16 features
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(480, 640, 3))  # Update input shape
vgg.trainable = False
output_layer = vgg.get_layer('block3_conv3').output
vgg_model = Model(vgg.input, output_layer)

def perceptual_loss(y_true, y_pred):
    y_true_features = vgg_model(y_true)
    y_pred_features = vgg_model(y_pred)
    return mean_squared_error(y_true_features, y_pred_features)

# Compile the model with Adam optimizer and perceptual loss
model.compile(optimizer=Adam(learning_rate=1e-4), loss=perceptual_loss)
# Train the model
model.fit(foggy_images, clear_images, batch_size=32, epochs=250, validation_split=0.1)

# Save the trained model
model.save('C:/Users/rahei/OneDrive/Desktop/archive/dehaze_model_Color_Saver_250_2.5.h5')


Epoch 1/250
1/1 [==============================] - 42s 42s/step - loss: 29.0447 - val_loss: 19.9386
Epoch 2/250


KeyboardInterrupt: 